In [24]:
from Bio.Entrez import efetch, read

def fetch_abstract(pmid):
    handle = efetch(db='pubmed', id=pmid, retmode='xml')
    xml_data = read(handle)['PubmedArticle'][0]
    try:
        article = xml_data['MedlineCitation']['Article']
        abstract = article['Abstract']['AbstractText'][0]
        return abstract
    except IndexError:
        return None

In [29]:
import pandas as pd
articles = pd.read_csv('extra/data/PubMed/oa_noncomm_xml.PMC001xxxxxx.baseline.2022-12-18.filelist.txt', delimiter='\t')

In [37]:
folder = 'extra/data/PubMed/text/'
for id in articles['PMID']:
    try:
        id = str(id)
        f = open(folder + id + '.txt', "x")
        f.write(fetch_abstract(id))
        f.close()
    except:
        continue


/Users/danielgui/opt/anaconda3/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(
